# 05-2. 교차 검증과 그리드 서치

In [2]:
#1
#테스트 세트로 일반화 성능을 올바르게 예측하려면 테스트 세트를 모델 평가에 가능한 쓰지말아야 함
#훈련세트 60%, 테스트세트 20%, 검증세트 20%로 데이터 나눔
#검증 세트 만들기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')


In [3]:
#2
#class열을 타깃으로 설정하고 나머지 열은 특성으로 사용
data = wine[['alcohol','sugar','pH']]
target = wine['class']

In [4]:
#3
#훈련 세트의 입력,타겟 데이터를 train_input, train_target에 저장
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
#4
#test size=0.2로 만듦. train_input의 약 20%를 val_input으로 만듬
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)


In [6]:
#5
#훈련세트와 검증세트의 크기를 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
#6
print(train_input.shape, test_input.shape)
#5197개 였던 훈련세트가 4157개로 줄고, test 세트는 1300개에서 1040개가 됨

(5197, 3) (1300, 3)


In [8]:
#7
#모델 만들고 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [9]:
#8
#교차검증-훈련세트를 쪼개서 모델평가를 한 다음 검증 점수 평균을 구하는 방법(몇개(K)fold)
#cross_validate라는 교차검증 함수를 사용
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0122757 , 0.00706267, 0.0071032 , 0.00674176, 0.00696158]), 'score_time': array([0.00194049, 0.00177932, 0.00146055, 0.00137329, 0.00132012]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [10]:
#9
#교차검증의 최종 점수는 test_score에 담긴 5개의 점수를 평균하여 얻을 수 있음
#이름은 test_score 이지만 검증 폴드의 점수임
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
#10
#사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해줌
from sklearn.model_selection import StratifiedKFold #타깃 클래스를 골고루 나누기 위해 사용
scores = cross_validate(dt,train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))
#cross_validate()함수는 회귀 모델일 경우 보통 KFold 분할기 사용

0.855300214703487


In [12]:
#11
#n_splits 매개변수는 몇 폴드 교차검증을 할지 정함
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt,train_input,train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


- 모델 파라미터-머신러닝 모델이 학습하는 파라미터
- 하이퍼 파라미터-사용자 지정 파라미터
- 하이퍼 파라미터를 자동으로 수행하는 기술을 Auto-ML
- 여러개의 매개변수의 최적값을 찾을 때 사이킷런에서 제공하는 그리드서치를 사용
- GridsearchCV : 하이퍼 파라미터 탐색과 교차 검증을 한번에 수행함. cross_validate 필요X


In [13]:
#12
#0.0001~0.0005 까지 증가하는 5개의 값 사용
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [14]:
#13
# GridsearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만듬
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)

In [15]:
#14
# 그리드서치 수행
gs.fit(train_input,train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'min_impurity_decrease': [0.0001, 0.0002, ...]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [16]:
#15
#훈련된 모델은 gs 객체의 best_estimator_속성에 있음
#점수 출력
dt = gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


In [17]:
#16
print(gs.best_params_)
#제일 좋은 값이 0.0001

{'min_impurity_decrease': 0.0001}


In [18]:
#17
#5번의 교차검증으로 얻은 점수를 출력
print(gs.cv_results_['mean_test_score'])
#test_score가 첫번째가 가장 커서 #16번 값이 나옴

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
#18
#최상의 검증 점수를 만든 매개변수 조합. 앞에서 출력한 #16과 값이 동일한지 확인
print(gs.cv_results_['params'][gs.best_index_])


{'min_impurity_decrease': 0.0001}


In [20]:
#19
#조금 더 복잡한 매개변수 조합 탐색
params = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001),#노드 분할하기 위한 불순도 감소 최소량 지정
          'max_depth' : range(5, 20, 1), #트리의 깊이 제한
          'min_samples_split' : range(2,100,10) #노드를 나누기 위한 최소 샘플 수 지정
          }

In [21]:
#20
#기본 5fold 교차 검증, 매개변수를 -1로 설정하고 그리드 서치 진행
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target) 

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'max_depth': range(5, 20), 'min_impurity_decrease': array([0.0001... 0.0009]), 'min_samples_split': range(2, 100, 10)}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [22]:
#21
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [23]:
#22
print(np.max(gs.cv_results_['mean_test_score']))
#GridSearchCV 클래스로 자동으로 교차검증을 수행해서 최상의 매개변수를 찾을 수 있음

0.8683865773302731


In [24]:
#23
#랜덤서치
#매개변수의 값이 수치이거나 많은 매개 변수 조건이 있어 그리스 서치 수행 시간이 오래 걸릴때 사용
from scipy.stats import uniform, randint

In [25]:
#24
#0~10 사이의 범위를 갖는 randint 객체를 만들고 10개의 숫자를 sampling
rgen = randint(0,10)
rgen.rvs(10)

array([1, 0, 8, 6, 4, 4, 2, 0, 7, 2])

In [26]:
#25
#sampling 개수를 1000개로 늘려 각 숫자의 개수를 세어봄
np.unique(rgen.rvs(1000), return_counts=True)
#1~9까지 숫자가 골고루 들어감

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([100, 105, 111,  95, 102, 101, 102,  82, 102, 100]))

In [27]:
#26
#uniform 클래스의 사용법, #25와 동일함. 0~1 사이에서 10개의 실수를 추출
ugen = uniform(0,1)
ugen.rvs(10)

array([0.14452075, 0.68180633, 0.4906978 , 0.47300066, 0.73431444,
       0.02887815, 0.93405035, 0.2385642 , 0.71912155, 0.67691147])

In [28]:
#27
#이제 랜덤서치에 randint, uniform 클래스 객체를 넘겨주고 최적의 매개변수 찾으라고 명령
#매개변수 탐색
params = {'min_impurity_decrease' : uniform(0.0001,0.001),#0.0001~0.001 사이의 실수값 샘플링
          'max_depth' : randint(20, 50), 
          'min_samples_split' : randint(2,25), #2~25 까지의 정수 샘플링
          'min_samples_leaf' : randint(1,25) #1~25 사이의 정수 샘플링
          }

In [31]:
#28
#사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                        n_iter =100, n_jobs=-1, random_state=42)
rs.fit(train_input,train_target)


,estimator,DecisionTreeC...ndom_state=42)
,param_distributions,"{'max_depth': <scipy.stats....x7db0f56d4b20>, 'min_impurity_decrease': <scipy.stats....x7db0f56d7b80>, 'min_samples_leaf': <scipy.stats....x7db0f56d66e0>, 'min_samples_split': <scipy.stats....x7db0f56d7340>}"
,n_iter,100
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [32]:
#29
#최적의 매개변수 조합 출력
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [33]:
#30
#최적의 교차검증 점수 출력
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [34]:
#31
#최적의 모델은 이미 전체 훈련세트로 훈련되어 best_estimator 속성에 저장되어 있음
#이 모델을 최종 모델로 결정하고 테스트 세트의 성능 확인
dt = rs.best_estimator_
print(dt.score(test_input,test_target))

0.86
